# **Detección de Armas - Open Data**  

📌 [**Información del Dataset**](https://dasci.es/es/transferencia/open-data/deteccion-de-armas/)  

Este proyecto se centra en la aplicación de **Data Augmentation** mediante **Diffusion Models** sobre este **conjunto de datos específico**. Nuestro objetivo es evaluar si estas técnicas pueden mejorar el rendimiento y los resultados en una tarea concreta de detección.

--- 
Este dataset cuenta con múltiples sub-datasets, donde nos encontramos con la siguiente estructura de directorios:

    Knife_classification  License.md             Pistol_detection  Weapons_and_similar_handled_objects/
    Knife_detection       Pistol_classification  README.md         pictures

Debido a las características del problema en concreto, solo trabajaremos con la parte de Object Detection (OD), es decir, las siguientes clases:

    Knife_detection                 
    Pistol_detection 
    Weapons_and_similar_handled_objects/Sohas_weapon-Detection-YOLOv5

Para empezar nos centraremos en:

    Weapons_and_similar_handled_objects/Sohas_weapon-Detection-YOLOv5

In [1]:
data_path = "/mnt/homeGPU/azapata/TFG/datasets/gunsUGR/OD-WeaponDetection-master/Weapons_and_similar_handled_objects/Sohas_weapon-Detection-YOLOv5/obj_train_data"
images_path = "/mnt/homeGPU/azapata/TFG/datasets/gunsUGR/OD-WeaponDetection-master/Weapons_and_similar_handled_objects/Sohas_weapon-Detection-YOLOv5/obj_train_data/images/train"
labels_path = "/mnt/homeGPU/azapata/TFG/datasets/gunsUGR/OD-WeaponDetection-master/Weapons_and_similar_handled_objects/Sohas_weapon-Detection-YOLOv5/obj_train_data/labels/train"

El dataset con el que estamos trabajando cuenta con:

- images
    - test ==> 857 images
    - train ==> 5002 images
- labels
    - test ==> 857 images
    - train ==> 5002 images


Vamos a cargar YOLO preentrenado y a evaluar sin finetunning directamente

In [ ]:
# # !pip install sympy
# # !pip install pandas>=1.1.4
# # !pip install seaborn>=0.11.0

# # !pip install opencv-python>=4.6.0
# # !pip install pandas>=1.1.4
# # !pip install seaborn>=0.11.0

# !pip install filelock
# !pip install networkx
# !pip install opencv-python>=4.6.0
# !pip install pandas>=1.1.4
# !pip install seaborn>=0.11.0


In [4]:
from ultralytics import YOLO

# Cargar el modelo preentrenado de YOLOv8
model = YOLO("yolov8l.pt")  

# Entrenar en tu dataset
model.train(
    data="/mnt/homeGPU/azapata/TFG/datasets/gunsUGR/OD-WeaponDetection-master/Weapons_and_similar_handled_objects/Sohas_weapon-Detection-YOLOv5/dataset.yaml",
    epochs=50,  # Ajusta según necesidad
    # batch=8,  # Ajusta según la GPU disponible
    # imgsz=640,  # Tamaño de imagen (ajusta si lo necesitas)
    device="cuda",  # Entrenamiento en GPU
    # workers=2,  # Número de hilos de carga de datos
    # optimizer="SGD",  # O prueba AdamW
    val=True  # Evalúa después de entrenar
)


In [ ]:
from ultralytics import YOLO

# Cargar el modelo entrenado
model = YOLO("/mnt/homeGPU/azapata/TFG/runs/detect/train/weights/best.pt")  # Ruta donde se guardó el modelo entrenado

# Evaluar en el conjunto de validación
metrics = model.val(
    data="/mnt/homeGPU/azapata/TFG/datasets/gunsUGR/OD-WeaponDetection-master/Weapons_and_similar_handled_objects/Sohas_weapon-Detection-YOLOv5/dataset.yaml", 
    split="val",
    device="cuda"
)

# Mostrar métricas clave
print(f"📊 Precisión (mAP@50): {metrics.box.map50:.4f}")
print(f"📊 Precisión (mAP@50-95): {metrics.box.map:.4f}")
print(f"📊 Precisión promedio por clase: {metrics.box.maps}")


Ultralytics 8.3.73 🚀 Python-3.8.20 torch-1.7.1+cu110 CUDA:0 (NVIDIA TITAN Xp, 12196MiB)
YOLOv8l summary (fused): 268 layers, 43,668,288 parameters, 0 gradients, 165.2 GFLOPs


val: Scanning /mnt/homeGPU/azapata/TFG/datasets/gunsUGR/OD-WeaponDetection-master/Weapons_and_similar_handled_objects/Sohas_weapon-Detection-YOLOv5/obj_train_data/labels/test.


val: New cache created: /mnt/homeGPU/azapata/TFG/datasets/gunsUGR/OD-WeaponDetection-master/Weapons_and_similar_handled_objects/Sohas_weapon-Detection-YOLOv5/obj_train_data/labels/test.cache


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 54/54 [00:19<00:00,  2.83it/s]


                   all        857        857   0.000744     0.0109   0.000387   0.000119
                person         85         85    0.00161     0.0588   0.000881   0.000378
               bicycle        140        140          0          0          0          0
                   car        452        452    0.00285    0.00664    0.00144   0.000336
            motorcycle         71         71          0          0          0          0
              airplane         52         52          0          0          0          0
                   bus         57         57          0          0          0          0
Speed: 0.7ms preprocess, 11.6ms inference, 0.0ms loss, 1.7ms postprocess per image
Results saved to /mnt/homeGPU/azapata/runs/detect/val10
📊 Precisión (mAP@50): 0.0004
📊 Precisión (mAP@50-95): 0.0001
📊 Precisión promedio por clase: [ 0.00037761           0   0.0003357           0           0           0  0.00011888  0.00011888  0.00011888  0.00011888  0.00011888  0.00011888 



Como es lógico, al no tener las mismas clases, los resultados son catastróficos. Vamos a pasar a hacer finetuning con el dataset para poder evaluar y obtener un baseline.


In [2]:
!python -c "import torch; print(torch.cuda.is_available())"

!python -c "import torch; print(torch.cuda.device_count())"
!python -c "import torch; print(torch.cuda.get_device_name(0))"

!nvcc --version

True
1
NVIDIA TITAN Xp
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0


---
# **Resultados Fine-Tuning de YOLO con las clases del dataset original**
Entrenamiento llevado a cabo con un script separado (TrainModels/Weapons/fineTunningOG.py)


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 215/215 [00:19<00:00, 10.89it/s]                   
                 all        857        857      0.848      0.789      0.886       0.73
                pistol         85         85       0.91      0.918      0.952      0.783
            smartphone        140        140      0.939      0.663      0.922      0.801
                 knife        452        452       0.97      0.903      0.973      0.676
              monedero         71         71      0.642      0.789      0.762       0.63
               billete         52         52      0.851      0.885      0.926      0.823
               tarjeta         57         57      0.777      0.579      0.782      0.665

---
# **Resultados Fine-Tuning de YOLO con las clases del dataset original con DA clásico**